### Import packages 

In [1]:
# Import all the necessary modules
from dagrad import dagrad # dagrad is the main class for learning the structure of a DAG
from dagrad import generate_linear_data, generate_nonlinear_data, count_accuracy

### Linear Model


In [2]:
n, d, s0, graph_type, sem_type = 1000, 10, 10, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
model = 'linear' # Define the model
W_notears = dagrad(X, model = model, method = 'notears') # Learn the structure of the DAG using Notears
W_dagma = dagrad(X, model = model, method = 'dagma') # Learn the structure of the DAG using Dagma
W_topo = dagrad(X, model = model, method = 'topo', loss_fn='user_loss') # Learn the structure of the DAG using Topo
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_notears = count_accuracy(B_true, W_notears != 0) # Measure the accuracy of the learned structure using Notears
print('Accuracy of Notears:', acc_notears)

acc_dagma = count_accuracy(B_true, W_dagma != 0) # Measure the accuracy of the learned structure using Dagma
print('Accuracy of Dagma:', acc_dagma)

acc_topo = count_accuracy(B_true, W_topo != 0) # Measure the accuracy of the learned structure using Topo
print('Accuracy of Topo:', acc_topo)


Using Original NOTEARS Implementation for Linear Model


/Users/duntrain/anaconda3/envs/test_dagrad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using Original DAGMA Implementation for Linear Model


100%|██████████| 180000/180000.0 [00:01<00:00, 170229.86it/s]


Parameter is automatically set up.
 size_small: 15, size_large: 28, no_large_search: 1


ValueError: Unknown combination of loss_fn and reg: user_loss, none. This combination is not supported. You can use other optimizers.

In [9]:
W_est = dagrad(X = X, 
               method = 'notears', 
               reg = 'l1', 
               optimizer = 'lbfgs', 
               general_options = {'tuning_method':'cv',
                                  'K':4, 
                                  'reg_paras':[0, 0.05, 0.1, 0.2, 0.5]})
acc = count_accuracy(B_true, W_est != 0)
print(acc)

Working with 4 fold cross validation on l1 with lambda1s: [0, 0.05, 0.1, 0.2, 0.5]
Optimal lambda1: 0.05
Retrain the model with optimal lambda1
Model retrained with optimal lambda1
{'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 10}


### Linear Model with different options (NOTEARS)

In [5]:
n, d, s0, graph_type, sem_type = 1000, 10, 10, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type) # Generate the data
model = 'linear' # Define the model

general_options = {'gamma':0.4, 'lambda1':0.1} # Define the general options
optimizer_options = {'lr':0.01,'num_steps':5000, 'check_iterate':500, 'tol':1e-5} # Define the optimizer options
method_options = {'verbose': False, 'rho':0.1} # Define the method options
# Learn the structure of the DAG using Notears
W_notears = dagrad(X, model = model, 
                   reg = 'mcp', 
                   optimizer= 'adam', 
                   method = 'notears', 
                   compute_lib='numpy',
                   general_options = general_options, 
                   optimizer_options = optimizer_options) 
acc_notears = count_accuracy(B_true, W_notears != 0) # Measure the accuracy of the learned structure using Notears
print('Accuracy of Notears:', acc_notears)

Accuracy of Notears: {'fdr': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 10, 'nnz': 0}


In [8]:
n, d, s0, graph_type, sem_type = 1000, 10, 10, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type) # Generate the data
model = 'linear' # Define the model
topo_init = list(range(d))

general_options = {'gamma':0.4, 'lambda1':0.1} # Define the general options
method_options = {'no_large_search':3, 'size_small': 10, 'size_large': 40,'topo':topo_init} # Define the method options

W_topo = dagrad(X, model = model, method = 'topo', loss_fn= 'l2', reg = 'mcp', optimizer = 'lbfgs', general_options=general_options, method_options=method_options) # Learn the structure of the DAG using Topo
acc_topo = count_accuracy(B_true, W_topo != 0) # Measure the accuracy of the learned structure using Topo
print('Accuracy of Topo:', acc_topo)

/Users/duntrain/Documents/Research/Code/Python/DAGlearner/dagrad/daglearner.py:162: UserWarning: For l2 or logistic loss, TOPO would be faster using optimizer: sklearn. But you are currently using optimizer: lbfgs.
  warn(f"For l2 or logistic loss, TOPO would be faster using optimizer: sklearn. But you are currently using optimizer: {optimizer}.")


Parameter is automatically set up.
 size_small: 10, size_large: 40, no_large_search: 3
Total Time: 3.2617831230163574
Accuracy of Topo: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 10}


### Nonlinear Model

In [23]:
n, d, s0, graph_type, sem_type = 1000, 10, 10, 'ER', 'mlp' # Define the parameters of the data
X, B_true = generate_nonlinear_data(n,d,s0,graph_type,sem_type,seed= 3) # Generate the data
model = 'nonlinear' # Define the model
W_notears = dagrad(X, model = model, method = 'notears') # Learn the structure of the DAG using Notears
W_dagma = dagrad(X, model = model, method = 'dagma') # Learn the structure of the DAG using Dagma
W_topo = dagrad(X, model = model, method = 'topo', general_options = {'lambda1':0.003,'lambda2':0}) # Learn the structure of the DAG using Topo
print(f'Nonlinear Model')
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_notears = count_accuracy(B_true, W_notears != 0) # Measure the accuracy of the learned structure using Notears
print('Accuracy of Notears:', acc_notears)

acc_dagma = count_accuracy(B_true, W_dagma != 0) # Measure the accuracy of the learned structure using Dagma
print('Accuracy of Dagma:', acc_dagma)

acc_topo = count_accuracy(B_true, W_topo != 0) # Measure the accuracy of the learned structure using Topo
print('Accuracy of Topo:', acc_topo)

Nonlinear Model
data size: 1000, graph type: ER, sem type: mlp
Accuracy of Notears: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 10}
Accuracy of Dagma: {'fdr': 0.09090909090909091, 'tpr': 1.0, 'fpr': 0.02857142857142857, 'shd': 1, 'nnz': 11}
Accuracy of Topo: {'fdr': 0.5833333333333334, 'tpr': 1.0, 'fpr': 0.4, 'shd': 14, 'nnz': 24}


In [21]:
W_topo = dagrad(X, model = model, method = 'topo', general_options = {'lambda1':0.003,'lambda2':0}) # Measure the accuracy of the learned structure using Topo
acc_topo = count_accuracy(B_true, W_topo != 0) # Measure the accuracy of the learned structure using Topo
print('Accuracy of Topo:', acc_topo)

Accuracy of Topo: {'fdr': 0.2, 'tpr': 0.8, 'fpr': 0.05714285714285714, 'shd': 4, 'nnz': 10}
